In [30]:
import re
import pandas as pd
import bs4
import requests
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

from spacy.matcher import Matcher 
from spacy.tokens import Span 

import networkx as nx

import matplotlib.pyplot as plt
from tqdm import tqdm

pd.set_option('display.max_colwidth', 200)
%matplotlib inline

In [31]:
# import wikipedia sentences
candidate_sentences = pd.read_csv(r"E:\Work\workbackups\llm\ayurveda_sample.csv")
candidate_sentences.shape

(15, 2)

In [32]:
candidate_sentences

,Unnamed: 0,sentence
0,0,Ayurveda is an ancient healing system from India
1,1,It means the science of life
2,2,It helps people to stay healthy and happy
3,3,"It uses natural methods like herbs, diet, yoga, and massage"
4,4,"It also considers the mind, body, and spirit of each person"
5,5,It says that everyone has a different type of energy or constitution
6,6,"There are three types of energy: vata, pitta, and kapha"
7,7,"They affect how a person thinks, feels, and acts"
8,8,They also affect the health and balance of the body
9,9,Ayurveda helps people to find out their type of energy and how to balance it


In [33]:
candidate_sentences.columns

Index(['Unnamed: 0', 'sentence'], dtype='object')

In [34]:
candidate_sentences = pd.DataFrame(candidate_sentences['sentence'])
candidate_sentences

,sentence
0,Ayurveda is an ancient healing system from India
1,It means the science of life
2,It helps people to stay healthy and happy
3,"It uses natural methods like herbs, diet, yoga, and massage"
4,"It also considers the mind, body, and spirit of each person"
5,It says that everyone has a different type of energy or constitution
6,"There are three types of energy: vata, pitta, and kapha"
7,"They affect how a person thinks, feels, and acts"
8,They also affect the health and balance of the body
9,Ayurveda helps people to find out their type of energy and how to balance it


In [35]:
candidate_sentences.shape

(15, 1)

In [36]:
def get_entities(sent):
  ## chunk 1
  ent1 = ""
  ent2 = ""

  prv_tok_dep = ""    # dependency tag of previous token in the sentence
  prv_tok_text = ""   # previous token in the sentence

  prefix = ""
  modifier = ""

  #############################################################
  
  for tok in nlp(sent):
    ## chunk 2
    # if token is a punctuation mark then move on to the next token
    if tok.dep_ != "punct":
      # check: token is a compound word or not
      if tok.dep_ == "compound":
        prefix = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          prefix = prv_tok_text + " "+ tok.text
      
      # check: token is a modifier or not
      if tok.dep_.endswith("mod") == True:
        modifier = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          modifier = prv_tok_text + " "+ tok.text
      
      ## chunk 3
      if tok.dep_.find("subj") == True:
        ent1 = modifier +" "+ prefix + " "+ tok.text
        prefix = ""
        modifier = ""
        prv_tok_dep = ""
        prv_tok_text = ""      

      ## chunk 4
      if tok.dep_.find("obj") == True:
        ent2 = modifier +" "+ prefix +" "+ tok.text
        
      ## chunk 5  
      # update variables
      prv_tok_dep = tok.dep_
      prv_tok_text = tok.text
  #############################################################

  return [ent1.strip(), ent2.strip()]

In [37]:
get_entities("the film had 200 patents")

['film', '200  patents']

In [38]:
entity_pairs = []

for i in tqdm(candidate_sentences["sentence"]):
  entity_pairs.append(get_entities(i))

100%|██████████| 15/15 [00:00<00:00, 166.62it/s]


In [39]:
def get_relation(sent):

  doc = nlp(sent)

  # Matcher class object 
  matcher = Matcher(nlp.vocab)

  #define the pattern 
  pattern = [{'DEP':'ROOT'}, 
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},  
            {'POS':'ADJ','OP':"?"}] 

  matcher.add("matching_1", [pattern], on_match = None)  

  matches = matcher(doc)
  k = len(matches) - 1

  span = doc[matches[k][1]:matches[k][2]] 

  return(span.text)

In [40]:
get_relation("John completed the task")

'completed'

In [41]:
relations = [get_relation(i) for i in tqdm(candidate_sentences['sentence'])]

100%|██████████| 15/15 [00:00<00:00, 169.65it/s]


In [42]:
# extract subject
source = [i[0] for i in entity_pairs]

# extract object
target = [i[1] for i in entity_pairs]

kg_df = pd.DataFrame({'source':source, 'target':target, 'edge':relations})

In [43]:
kg_df

,source,target,edge
0,Ayurveda,ancient healing India,is
1,It,life,means
2,people,,helps
3,It,natural herbs,uses natural
4,It,also person,considers
5,everyone,different energy,says
6,,three energy,are
7,how person,,affect
8,They,also body,affect
9,people,how it,helps
